In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [ ]:
import time
import os
from aiida import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()
from aiida.orm import Code, load_node
from aiida.orm import DataFactory, CalculationFactory
from aiida_kkr.tools.kkrcontrol import write_kkr_inputcard_template, fill_keywords_to_inputcard, create_keyword_default_values
from pprint import pprint
from scipy import array
from aiida_kkr.calculations.kkr import KkrCalculation
from aiida_kkr.calculations.voro import VoronoiCalculation
from aiida_kkr.parsers.voro import VoronoiParser
from aiida_kkr.parsers.kkr import KkrParser


#from aiida_kkr.tools.kkrcontrol import 
# example_fcc_Cu_lmax2_ASA_noSOC
ParameterData = DataFactory('parameter')
StructureData = DataFactory('structure')

In [ ]:
alat = 6.830000 # in a_Bohr
abohr = 0.52917721067
# number of atom positions in unit cell
natyp = 1
# bravais vectors
bravais = array([[0.5, 0.5, 0.0], [0.5, 0.0, 0.5], [0.0, 0.5, 0.5]])
print bravais.max()
keywords = create_keyword_default_values()
keywords['NATYP'][0] = natyp
keywords['ALATBASIS'][0] = alat
keywords['NSPIN'][0] = 1
keywords['LMAX'][0] = 2
# choose only coarse energy contour and k-mesh for test purposes
keywords['NPOL'][0] = 4
keywords['NPT1'][0] = 3
keywords['NPT2'][0] = 10
keywords['NPT3'][0] = 3
keywords['BZKX'][0] = 10
keywords['BZKY'][0] = 10
keywords['RCLUSTZ'][0] = 1.50
keywords['RCLUSTXY'][0] = 1.50
# for ASA
keywords['INS'] = [0, '%i']
keywords['KSHAPE'] = [0, '%i']

In [ ]:
1/abohr

In [ ]:
pprint(keywords)

In [ ]:
keyw = ParameterData(dict=keywords)
#keyw.store()
keyw = load_node(79550)
print keyw

In [ ]:
a = 0.5*alat*abohr
Cu = StructureData(cell=[[a, a, 0.0], [a, 0.0, a], [0.0, a, a]])
Cu.append_atom(position=[0.0, 0.0, 0.0], symbols='Cu')
#Cu.store()
Cu = load_node(79546)
print(Cu)

In [ ]:
from scipy import array
cell = array(Cu.cell)
print cell
sites = Cu.sites

In [ ]:
kinds = Cu.kinds
print Cu.get_kind('Cu')
print kinds
for site in sites:
    print(site.kind_name)
    sitekind = Cu.get_kind('Cu')
    symbol = sitekind.symbol
    print symbol
    a = site.position
    print a

In [ ]:
#CalculationFactory('kkr.voro')

In [ ]:
! verdi calculation plugins

In [ ]:
# vornoi
code = Code.get_from_string('voro@local_mac')#'kkrimp@local_mac')
calc = VoronoiCalculation()#KkrCalculation()
calc.label = 'Test kkr'
calc.set_withmpi(False)
calc.set_resources({"num_machines" : 1})
calc.set_max_wallclock_seconds(300)
calc.set_computer('local_mac')
calc.use_code(code)
calc.use_structure(Cu)
calc.use_parameters(keyw)

In [ ]:
submit_test = True

In [ ]:
if submit_test:
    subfolder, script_filename = calc.submit_test()
    print "Test_submit for calculation (uuid='{}')".format(
        calc.uuid)
    print "Submit file in {}".format(os.path.join(
        os.path.relpath(subfolder.abspath),
        script_filename
    ))
else:
    calc.store_all()
    print "created calculation; calc=Calculation(uuid='{}') # ID={}".format(
        calc.uuid, calc.dbnode.pk)
    calc.submit()
    print "submitted calculation; calc=Calculation(uuid='{}') # ID={}".format(
        calc.uuid, calc.dbnode.pk)

In [ ]:
! cat submit_test/20171110-00007/_aiidasubmit.sh
!ls submit_test/20171110-00001/

In [ ]:
! cat submit_test/20171110-00007/inputcard

In [ ]:
n = load_node(79559)
retrieved_dict = {'retrieved' : n}

In [ ]:
voro_parser = VoronoiParser(calc)

In [ ]:
suc, nodes = voro_parser.parse_with_retrieved(retrieved_dict)

In [ ]:
print suc
print nodes

In [ ]:
print nodes[0][1].get_dict()['EMIN']

In [ ]:
calc2 = load_node(79565)

In [ ]:
#emin = calc1.get_outputs_dict()['output_parameters'].get_dict()['EMIN']
emin = calc2.res.EMIN
remote = calc2.get_outputs_dict()['remote_folder']
keywords2 = keywords
keywords2['EMIN'][0] = emin
#pprint(keywords2)

In [ ]:
keyw2 = ParameterData(dict=keywords2)
#keyw2.store()
keyw2 = load_node(79570)
print keyw2

In [ ]:
isinstance(calc, VoronoiCalculation)

In [ ]:
# kkr
code = Code.get_from_string('kkr1@local_mac')#'kkrimp@local_mac')
calc1 = KkrCalculation()
calc1.label = 'Test kkr'
calc1.set_withmpi(False)
calc1.set_resources({"num_machines" : 1})
calc1.set_max_wallclock_seconds(300)
calc1.set_computer('local_mac')
calc1.use_code(code)
#calc1.use_structure(Cu)
calc1.use_parameters(keyw2)
calc1.use_parent_folder(remote)

In [ ]:
from aiida.orm import JobCalculation
remote.get_inputs(node_type=JobCalculation)

In [ ]:
submit_test = False

In [ ]:
if submit_test:
    subfolder, script_filename = calc1.submit_test()
    print "Test_submit for calculation (uuid='{}')".format(
        calc1.uuid)
    print "Submit file in {}".format(os.path.join(
        os.path.relpath(subfolder.abspath),
        script_filename
    ))
else:
    calc1.store_all()
    print "created calculation; calc=Calculation(uuid='{}') # ID={}".format(
        calc1.uuid, calc.dbnode.pk)
    calc1.submit()
    print "submitted calculation; calc=Calculation(uuid='{}') # ID={}".format(
        calc.uuid, calc.dbnode.pk)

In [ ]:
!pwd

In [ ]:
! cat submit_test/20171110-00020///_aiidasubmit.sh
! ls submit_test/20171110-00020//
! cat submit_test/20171110-00020/inputcard